In [1]:
# setting device on GPU if available, else CPU
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')


Using device: cuda

NVIDIA GeForce RTX 2080 Ti
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
import wandb

import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.utils.data import DataLoader

from torch.utils import data


import copy
import os
import random
import cv2
import numpy as np
from PIL import Image
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob
import functools
from tqdm import tqdm
from datetime import datetime
import numpy as np
# from core.datasets.vqa_motion_dataset import VQMotionDataset,DATALoader,VQVarLenMotionDataset,MotionCollator
from einops import rearrange, reduce, pack, unpack
import sys

In [4]:
from typing import Optional, List, Dict, Tuple

In [12]:
from configs.config import cfg, get_cfg_defaults
from core.datasets.dataset_loading_utils import load_dataset
from core.datasets.vq_dataset import DATALoader
from utils.vis_utils import plot_3d_global
from core.models.conv_vqvae import ConvVQMotionModel
import utils.rotation_conversions as geometry

In [6]:
cfg = get_cfg_defaults()
cfg.merge_from_file("/srv/hays-lab/scratch/sanisetty3/music_motion/TGM3D/checkpoints/conformer_512_1024_affine/conformer_512_1024_affine.yaml")

In [7]:
cfg.vqvae.motion_dim

135

In [8]:
vqvae_model = ConformerVQMotionModel(
            cfg.vqvae,
            # is_distributed=self.is_distributed,
        )

Sync is turned on False


## Dataloader

In [9]:
render_ds, sampler_val, weights_val = load_dataset(
                ["t2m", "aist", "cm"],cfg, "render"
            )

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 703.68it/s]


Total number of motions 2


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 559.24it/s]


Total number of motions 2


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 368.01it/s]

Total number of motions 3


In [10]:
render_dl = DATALoader(
    
            render_ds, batch_size=2, shuffle=False
        )

In [22]:
batch = (next(iter(render_dl)))
    
print(batch["names"])

['00_000048']


In [6]:
from utils.vis_utils.render_final import Renderer
renderer = Renderer("cuda")

In [14]:
from pathlib import Path


In [43]:
def load(vqvae_model,  path):
    path = Path(path)
    assert path.exists()
    pkg = torch.load(str(path), map_location="cuda")
    vqvae_model.vq._codebook.batch_mean = pkg["model"][
        "vq._codebook.batch_mean"
    ]
    vqvae_model.vq._codebook.batch_variance = pkg["model"][
        "vq._codebook.batch_variance"
    ]

    vqvae_model.load_state_dict(pkg["model"])
    vqvae_model = vqvae_model.to(device)

In [6]:
from core.models.conformer_vqvae import ConformerVQMotionModel

In [7]:
cfg = get_cfg_defaults()
cfg.merge_from_file("/srv/hays-lab/scratch/sanisetty3/music_motion/TGM3D/checkpoints/conformer_512_1024_affine/conformer_512_1024_affine.yaml")

In [9]:
model = ConformerVQMotionModel(cfg.vqvae)

Sync is turned on False


In [16]:
save_path = "/srv/hays-lab/scratch/sanisetty3/music_motion/TGM3D/checkpoints/conformer_512_1024_affine/checkpoints/vqvae_motion.320000.pt"

In [44]:
load(model,save_path)
model = model.eval()

In [79]:
aa = model.motionEncoder( torch.randn((1, 80 , 271)).cuda() , True)

In [92]:
embed_motion_features = aa.mean() + torch.randn((1, 20 , 768)).to(model.device) * aa.var()

In [93]:
quantized_enc_motion, indices, commit_loss = model.vq(embed_motion_features)

In [94]:
decoded_motion_features = model.motionDecoder(quantized_enc_motion, True)

In [7]:
render_path = "/srv/hays-lab/scratch/sanisetty3/music_motion/TGM3D/renders/"

In [8]:
data  = np.load('/coc/scratch/sanisetty3/music_motion/GTA/yogi.npz')

In [17]:
data["poses"][:,:66].shape

(892, 66)

In [23]:
data["trans"].shape

(892, 3)

In [22]:
rots = geometry.matrix_to_rotation_6d(geometry.axis_angle_to_matrix(torch.Tensor(data["poses"][:,:66].reshape(-1,22,3)))).reshape(-1,132)

In [28]:
renderer.render(
                    motion_vec=torch.Tensor(np.concatenate([data["trans"] , rots.numpy()] , 1)),
                    outdir=render_path,
                    step=0,
                    name="rnd2",
                )

## New LOss

In [11]:
from core.models.smpl.body_model import BodyModel
import utils.rotation_conversions as geomtry

In [12]:
bm = BodyModel("/srv/hays-lab/scratch/sanisetty3/music_motion/motion-diffusion-model/body_models/smplh/neutral/model.npz")

In [13]:
for batch in render_dl:
    break

In [14]:
m6d = batch["motion"][:,:,:135]

In [15]:
m6d.shape

torch.Size([2, 80, 135])

In [16]:
bs , n , d = m6d.shape

In [76]:
root_trans = 

In [95]:
aa = geomtry.matrix_to_axis_angle(geomtry.rotation_6d_to_matrix(m6d[:,:,3:135].reshape(-1,22,6))).reshape(-1 , 66)

In [96]:
aa.shape

torch.Size([160, 66])

In [98]:
body = bm.forward(root_orient=aa[:,:3] , pose_body = aa[:,3:] , trans=m6d[:,:,:3].reshape(-1,3) , return_dict = True)

In [99]:
body.keys()

dict_keys(['v', 'f', 'Jtr', 'full_pose'])

In [103]:
body["Jtr"].reshape(bs , n , 52,3).shape

torch.Size([2, 80, 52, 3])

In [100]:
(body["Jtr"][1:] - body["Jtr"][:-1]).shape

torch.Size([159, 52, 3])

In [85]:
Loss = torch.nn.SmoothL1Loss()

In [86]:
Loss(body["Jtr"] , body["Jtr"])

tensor(0.)

In [113]:
m6d.shape

torch.Size([2, 80, 271])

In [20]:
def forward_new(motion_pred, motion_gt, mask=None):
    Loss = torch.nn.SmoothL1Loss()
    bs, n, d = motion_gt.shape  ## d = 135
    motion_pred_aa = geomtry.matrix_to_axis_angle(
        geomtry.rotation_6d_to_matrix(motion_pred[:, :, 3:135].reshape(-1, 22, 6))
    ).reshape(-1, 66)
    body_pred = bm.forward(
        root_orient=motion_pred_aa[:, :3],
        pose_body=motion_pred_aa[:, 3:],
        trans=motion_pred[:, :, :3].reshape(-1, 3),
        return_dict=True,
    )

    motion_gt_aa = geomtry.matrix_to_axis_angle(
        geomtry.rotation_6d_to_matrix(motion_gt[:, :, 3:135].reshape(-1, 22, 6))
    ).reshape(-1, 66)
    body_gt = bm.forward(
        root_orient=motion_gt_aa[:, :3],
        pose_body=motion_gt_aa[:, 3:],
        trans=motion_gt[:, :, :3].reshape(-1, 3),
        return_dict=True,
    )

    j_pred = body_pred["Jtr"].reshape(bs, n, 52, 3)[:, : 22, :]
    j_gt = body_gt["Jtr"].reshape(bs, n, 52, 3)[:, : 22, :]

    loss_pos = Loss(
        j_pred,
        j_gt,
    )
    loss_vel = Loss(
        (j_pred[:, 1:] - j_pred[:, :-1]), (j_gt[:, 1:] - j_gt[:, :-1])
    )

    return loss_pos, loss_vel

In [23]:
motion_pred, _ , _ = vqvae_model(m6d[:, :, :135])

In [24]:
motion_pred.shape

torch.Size([2, 80, 135])

In [25]:
forward_new(motion_pred ,m6d )

(tensor(0.1199, grad_fn=<SmoothL1LossBackward0>),
 tensor(0.0787, grad_fn=<SmoothL1LossBackward0>))

In [29]:
np.load("../HumanMotionSMPL/Mean.npy")[

array([ 2.42843907e-02,  1.13441283e+00, -8.98784186e-03, -2.40984987e-03,
       -6.52230864e-04, -9.92232620e-04, -6.47345805e-03,  9.72569679e-01,
        1.95284334e-02,  9.65837892e-01, -1.06665792e-01, -2.10577656e-02,
        1.12139878e-01,  8.77811686e-01,  2.35941030e-01,  9.67205206e-01,
        8.93590907e-02,  2.49379556e-02, -9.78139769e-02,  8.75245933e-01,
        2.57273114e-01,  9.84242926e-01, -1.86104024e-02,  8.78357464e-03,
        1.86940623e-02,  9.51855687e-01, -1.40547394e-01,  9.64632180e-01,
        6.87693139e-02, -4.39669236e-02, -6.46595745e-02,  7.50103124e-01,
       -4.13668740e-01,  9.68300201e-01, -5.46530094e-02,  8.35328890e-03,
        4.05742119e-02,  7.39468683e-01, -4.42177031e-01,  9.91846624e-01,
       -2.63669251e-03, -1.29083900e-02,  2.03791298e-03,  9.82947577e-01,
       -6.16558589e-02,  9.84936983e-01, -4.41092218e-04,  6.37313024e-02,
       -1.02082998e-02,  9.64401329e-01,  1.14246273e-01,  9.86655531e-01,
       -2.35986068e-02, -

In [29]:
pwd

'/coc/scratch/sanisetty3/music_motion/TGM3D'

In [31]:
cd PyMAF-X/

/coc/scratch/sanisetty3/PyMAF-X


In [ ]:
!python -m apps.demo_smplx --image_folder examples/coco_images --detection_threshold 0.3 --pretrained_model data/pretrained_model/PyMAF-X_model_checkpoint_v1.1.pt --misc TRAIN.BHF_MODE full_body MODEL.PyMAF.HAND_VIS_TH 0.1